## Spatial transcriptomics preprocessing - single-cell Xenium data

In these GettingStarted notebooks, we will guide you through the process of preprocessing, training, and performing inference with DeepCell on your single-cell spatial transcriptomics data (Xenium). First, we will begin with data preprocessing. In the training notebook, we will demonstrate how to train DeepCell, adjust hyperparameters, and export the model weights. Finally, in the inference notebook, we will show you how to load the model weights and perform single-cell spatial transcriptomics prediction using H&E images.

In [ ]:
import os
os.chdir('../../')

Export packages

In [ ]:
from deepspot.utils.utils_image import get_morphology_model_and_preprocess
from deepspot.utils.utils_image import crop_tile
from pathlib import Path
from tqdm import tqdm
import scanpy as sc
import pandas as pd
import numpy as np
import pyvips
import torch
import glob
import yaml
import json

Specify the input files. For this example, we have selected one Xenium sample from the 'Image-based spatial transcriptomics identifies molecular niche dysregulation associated with distal lung remodeling in pulmonary fibrosis' dataset [1], which was downloaded using the HEST1K pipeline [2]. You can modify this pipeline to process multiple samples. The goal is to help you understand the underlying logic.

[1] Vannan, A., Lyu, R., Williams, A.L., Negretti, N.M., Mee, E.D., Hirsh, J., Hirsh, S., Hadad, N., Nichols, D.S., Calvi, C.L. and Taylor, C.J., 2025. Spatial transcriptomics identifies molecular niche dysregulation associated with distal lung remodeling in pulmonary fibrosis. Nature Genetics, pp.1-12.

[2] Jaume, G., Doucet, P., Song, A. H., Lu, M. Y., Almagro-Pérez, C., Wagner, S. J., ... & Mahmood, F. (2024). Hest-1k: A dataset for spatial transcriptomics and histology image analysis. arXiv preprint arXiv:2406.16192.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
image_feature_model = "inception" # foundation model 
sample = "NCBI858" # Lung cancer dataset sample
out_folder = "example_data"
adata_in = f"example_data/data/h5ad/{sample}.h5ad"
json_path = f"example_data/data/meta/{sample}.json"
img_path = f"example_data/data/image/{sample}.jpg"

In [ ]:
cell_diameter = 20 # cell diameter
cell_diameter

In [ ]:
# create folder to save tile embeddings
folder_to_create = f"{out_folder}/data/image_features/{image_feature_model}_{cell_diameter}/{sample}"

Path(folder_to_create).mkdir(parents=True, exist_ok=True)

Path(f"{out_folder}/data/inputX").mkdir(parents=True, exist_ok=True)

We start by loading the spatial transcriptomics data and selecting the most variable genes

In [ ]:
adata = sc.read_h5ad(adata_in)
adata

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='seurat_v3_paper', 
                            n_top_genes=500)
adata

We select the most highly variable genes in the isPredicted variable and save this table. Later, it is useful to understand which genes are the most variable (e.g., based on their rank).

In [ ]:
adata.var["isPredicted"] = adata.var.highly_variable.values
adata.var["gene_name"] = adata.var.index.values
adata.var

In [ ]:
adata.var.to_csv(f"{out_folder}/data/info_highly_variable_genes_Xenium.csv", index=False)

Here, we save only the counts and store them as a pickle file to make them quickly accessible during training.

In [ ]:
counts = pd.DataFrame(adata.X, index=adata.obs_names.values, columns=adata.var.index)
counts.index = [f"{b}_{sample}" for b in counts.index]
counts

In [ ]:
counts.to_pickle(f"{out_folder}/data/inputX/{sample}.pkl")

We load the pathology foundation model along with its preprocessing pipeline and feature dimensions. Currently, we support Phikon, Uni, DenseNet121, ResNet50, and Inception. However, this list can be extended to include more models by modifying the `get_morphology_model_and_preprocess` function.

In [ ]:
morphology_model, preprocess, feature_dim = get_morphology_model_and_preprocess(model_name=image_feature_model, 
                                                                                device=device)
feature_dim

We now begin extracting tile representations and store them to save time during training by using the precomputed data. The representations are stored per spot, which includes the main representation and the k non-overlapping subspots, resulting in a shape of `(n_spots, k+1, feature_dim)`.

In [ ]:
image = pyvips.Image.new_from_file(img_path)
morphology_model = morphology_model.to(device)
barcode = adata.obs_names
x_pixel = adata.obs.x_pixel
y_pixel = adata.obs.y_pixel


image = pyvips.Image.new_from_file(img_path)

In [ ]:
main_features = []

batch_X = []
batch_barcode = []
batch_size = 128


for i, (b, x, y) in tqdm(enumerate(zip(barcode, x_pixel, y_pixel))): 
    
    patch = crop_tile(image, x, y, cell_diameter)
    X = preprocess(patch)
    
    X = X.unsqueeze(0)
    X = X.to(device)
    X = X.float()

    # Accumulate the preprocessed patches into a batch
    batch_X.append(X)
    batch_barcode.append(b)
    if len(batch_X) == batch_size:
        batch_tensor = torch.cat(batch_X, dim=0)  # Combine the list of tensors into one tensor
        
        
        # We recommend using mixed precision for faster inference.
        with torch.autocast(device_type="cuda", dtype=torch.float32):
            with torch.inference_mode():
                output = morphology_model(batch_tensor)
                output = output.detach().cpu().numpy()

        for b, embedding in zip(batch_barcode, output):
            np.save(f"{folder_to_create}/{b}.npy", embedding)
        
        batch_barcode.clear() # Reset the batch list
        batch_X.clear()  # Reset the batch list

# Process any remaining patches if they exist
if batch_X and batch_barcode:
    batch_tensor = torch.cat(batch_X, dim=0)
    batch_X.clear()  # Reset the batch list
    
    # We recommend using mixed precision for faster inference.
    with torch.autocast(device_type="cuda", dtype=torch.float32):
        with torch.inference_mode():
            output = morphology_model(batch_tensor)
            output = output.detach().cpu().numpy()
                
    for b, embedding in zip(batch_barcode, output):
        np.save(f"{folder_to_create}/{b}.npy", embedding)

Each spot is encoded with its unique barcode per slide id and the slide id itslef.

In [ ]:
glob.glob(f"{out_folder}/data/image_features/{image_feature_model}_{cell_diameter}/{sample}/*")[:10]